In [1]:
import numpy as np
import wandb
from collections import OrderedDict

import cv2
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from test_data.factory_test import dataset_factory
from config import cfg

import lietorch
from lietorch import SE3
from geom.losses import geodesic_loss
# from src.geom.losses import geodesic_loss

# network
# from src.model import ViTEss
from logger import Logger
from cuti import build

# DDP training
import torch.multiprocessing as mp
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP

from tqdm import tqdm
import random
from datetime import datetime
import os

import torchvision.models as models

from cuti import build
from config import cfg

In [2]:
db = dataset_factory(
    ["matterport"],
    datapath="/home/kmuvcl/source/CuTi/matterport",
    subepoch=0,
    is_training=False,
    gpu = 0,
    streetlearn_interiornet_type = None,
    use_mini_dataset = False
)

Building Matterport dataset
Dataset matterport has 7996 images


In [3]:
model = build(cfg)

In [4]:
ckpt = "/home/kmuvcl/source/CuTi/checkpoint/120000.pth"

In [5]:
state_dict = OrderedDict([
        (k.replace("module.", ""), v) for (k, v) in torch.load(ckpt)['model'].items()])

In [6]:
model.load_state_dict(state_dict,False)
model = model.cuda().eval()

In [7]:

test_loader = DataLoader(
 db, batch_size=cfg.SOLVER.BATCH_SIZE, num_workers=1, shuffle=False
)

In [19]:
DEPTH_SCALE =5
predictions = {'camera': {'preds': {'tran': [], 'rot': []}, 'gts': {'tran': [], 'rot': []}}}
loss_list =[]

In [23]:
def eval_camera(predictions):
    acc_threshold = {
        "tran": 1.0,
        "rot": 30,
    }  # threshold for translation and rotation error to say prediction is correct.

    pred_tran = np.vstack(predictions["camera"]["preds"]["tran"])
    pred_rot = np.vstack(predictions["camera"]["preds"]["rot"])

    gt_tran = np.vstack(predictions["camera"]["gts"]["tran"])
    gt_rot = np.vstack(predictions["camera"]["gts"]["rot"])

    top1_error = {
        "tran": np.linalg.norm(gt_tran - pred_tran, axis=1),
        "rot": 2 * np.arccos(np.clip(np.abs(np.sum(np.multiply(pred_rot, gt_rot), axis=1)), -1.0, 1.0)) * 180 / np.pi,
    }
    top1_accuracy = {
        "tran": (top1_error["tran"] < acc_threshold["tran"]).sum()
        / len(top1_error["tran"]),
        "rot": (top1_error["rot"] < acc_threshold["rot"]).sum()
        / len(top1_error["rot"]),
    }
    camera_metrics = {
        f"top1 T err < {acc_threshold['tran']}": top1_accuracy["tran"] * 100,
        f"top1 R err < {acc_threshold['rot']}": top1_accuracy["rot"] * 100,
        f"T mean err": np.mean(top1_error["tran"]),
        f"R mean err": np.mean(top1_error["rot"]),
        f"T median err": np.median(top1_error["tran"]),
        f"R median err": np.median(top1_error["rot"]),
    }
    
    gt_mags = {"tran": np.linalg.norm(gt_tran, axis=1), "rot": 2 * np.arccos(gt_rot[:,0]) * 180 / np.pi}

    tran_graph = np.stack([gt_mags['tran'], top1_error['tran']],axis=1)
    tran_graph_name = os.path.join('output', args.exp, output_folder, 'gt_translation_magnitude_vs_error.csv')
    np.savetxt(tran_graph_name, tran_graph, delimiter=',', fmt='%1.5f')

    rot_graph = np.stack([gt_mags['rot'], top1_error['rot']],axis=1)
    rot_graph_name = os.path.join('output', args.exp, output_folder, 'gt_rotation_magnitude_vs_error.csv')
    np.savetxt(rot_graph_name, rot_graph, delimiter=',', fmt='%1.5f')
    
    return camera_metrics

In [24]:
with tqdm(test_loader, unit="batch") as tepoch:
    for i_batch, item in enumerate(tepoch):
        images, poses, intrinsics, lines = [x.to("cuda") for x in item]
        Ps = SE3(poses) 
        Gs = SE3.IdentityLike(Ps)
        Ps_out = SE3(Ps.data.clone())
        with torch.no_grad():
            poses_est = model(images, lines, Gs)
        #print(poses.shape)
        #print(poses[0][1])
        for i in range(6):
            predictions['camera']['gts']['tran'].append(poses[i][1][:3])
            gt_rotation = poses[i][1][3:]
            if gt_rotation[0] < 0: # normalize quaternions to have positive "W" (equivalent)
                gt_rotation[0] *= -1
                gt_rotation[1] *= -1
                gt_rotation[2] *= -1
                gt_rotation[3] *= -1
            predictions['camera']['gts']['rot'].append(gt_rotation)
            
            preds = poses_est[0][i][1].data.cpu().numpy()
            
            pr_copy = np.copy(preds)
            
            preds[3] = pr_copy[6] # swap 3 & 6, we used W last; want W first in quat
            preds[6] = pr_copy[3]
            preds[:3] = preds[:3] * DEPTH_SCALE 
            
            predictions['camera']['preds']['tran'].append(preds[:3])
            predictions['camera']['preds']['rot'].append(preds[3:])
            
            


        
        

 30%|███       | 400/1333 [03:06<07:36,  2.04batch/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f0489070dc0>
Traceback (most recent call last):
  File "/home/kmuvcl/anaconda3/envs/cuti/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__
    self._shutdown_workers()
  File "/home/kmuvcl/anaconda3/envs/cuti/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
    if w.is_alive():
  File "/home/kmuvcl/anaconda3/envs/cuti/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f0489070dc0>
Traceback (most recent call last):
  File "/home/kmuvcl/anaconda3/envs/cuti/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__
    self._shutdown_workers()
  File "

RuntimeError: mat1 dim 1 must match mat2 dim 0

In [ ]:
print(predictions)

In [26]:
camera_metrics = eval_camera(predictions)

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

: 

In [ ]:
for k in camera_metrics:
    print(k, camera_metrics[k])